# Topic modelling of News articles - Farmer suicides

In [0]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import csv


import requests
from bs4 import BeautifulSoup
import urllib.request
from threading import Thread
import urllib
import re
import time
import pandas as pd


from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim




links=[]
all_posts=[]
link_post =[]
Date=[]
Head_line=[]
Dates=[]
Paper=[]


# Input topic to scrape relevant news articles

In [0]:
class Scraper(object):
    def __init__(self):
    
        options = webdriver.ChromeOptions() 

        self.driver = webdriver.Chrome()
        
        try:
            url = 'https://www.google.com/search?q='+str(input()+"&tbm=nws&num=100")
        except Exception as e:
            print(str(e))
        self.driver.get(url)
        self.driver.refresh()
        time.sleep(1)

       
    def next_page(self):
        try:
            butn= self.driver.find_element_by_id('pnnext')
            butn.click()
            self.extract_post_information()
            
        except:
            print("end of search, no of searches="+str(len(links)))
            Date=[(str(w).strip('[,]')).split('-',1) for w in links]
            for i in range(len(Date)):
                try:
                    Dates.append(Date[i][1])
                    Paper.append(Date[i][0])
                except:
                    Dates.append(Date[i])
                    Paper.append(Date[i])
            self.driver.close()
           
    def extract_post_information(self):
        
        all_posts = self.driver.find_elements_by_class_name("g")
        
        for a in self.driver.find_elements_by_xpath('//*[@id="rso"]/div/div/div/div/h3/a'):
            link_post.append(a.get_attribute('href'))
        
        title=[]
        for post in all_posts:
            title=post.text.split("\n")
            Head_line.append((str(title[0:1]).replace("'", "")).strip('[,]'))
            links.append((str(title[1:2]).replace("'", "")).strip('[,]'))

        self.next_page()
#         print("url= "+str(links)+"\n")

    
    
scraper = Scraper()
# scraper.setting_100()

scraper.extract_post_information()
# scraper.next_page()

cotton suicides india
end of search, no of searches=878


In [0]:
len(Dates)

883

# Saving csv

In [0]:
Dates=[(str(w).replace("'", "")).strip('[,]') for w in Dates]
Paper=[(str(w).replace("'", "")).strip('[,]') for w in Paper]
News = {"Dates": Dates[:878], "Paper": Paper[:878],"Head_line": Head_line ,"link_post": link_post }
df   = pd.DataFrame(News, columns=['Dates','Paper','Head_line','link_post'])
df.to_csv('/Users/ipsita.praharaj1997/Desktop/cotton suicides india.csv', index=False)

In [0]:
# df

# Setting timer to scrape article body

In [0]:
df = pd.read_csv('/Users/ipsita.praharaj1997/Desktop/cotton suicides india.csv')
counts = df['Paper'].value_counts()
fkeys=[]
fvalues=[]
for fkey, fvalue in counts.items():
    fkeys.append(fkey)
    fvalues.append(fvalue)
    
df2=pd.DataFrame( { "title": fkeys, "frequency": fvalues  } )

start=[]
stop=[]
timer= []

k=1

for i in range(len(df2)):
    start.append(k)
    stop.append(k+df2.loc[i,'frequency'])
    k= k+df2.loc[i,'frequency']
    
#TIME SETTER FOR LARGE SET OF LINKS

timer= list( map( lambda x: 5 if x> 10 else 1,df2['frequency'] )) 

df2['start']=start
df2['stop']=stop
df2['second']=timer


In [0]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [0]:
df2.to_csv('/Users/ipsita.praharaj1997/Desktop/Frequency-cotton suicides india.csv',index=False)

# Scraping content of links using Multithreading

In [0]:
sec_ar=[]
start_ar=[]
stop_ar=[]
paper=[]
start=0
stop=0
sec=0
paper=''

df=pd.read_csv('/Users/ipsita.praharaj1997/Desktop/Frequency-cotton suicides india.csv')

sec_ar = df['second']
start_ar = df['start']
stop_ar = df['stop']
paper_ar = df['title']

In [0]:
#READING FROM THE DATESET  #####################################

df1= pd.read_csv('/Users/ipsita.praharaj1997/Desktop/cotton suicides india.csv')
corpus_link = df1['link_post']
threadlist=[]
df1['keys']=df1.index

sad=[]

for i in range(10):
    start = start_ar[i]
    stop = stop_ar[i]
    sec = sec_ar[i]
    paper = paper_ar[i]
    
#THREADING  ##################################################
   
    corpus_dict = {}
    def th(ur, key):

        try:
            time.sleep(sec)
            response = requests.get(ur).text
            time.sleep(sec/2)
            soup = BeautifulSoup(response, "lxml")
            review_texts = soup.findAll('p') #########################  VARIES FROM PAPER TO PAPER
        #     review_texts = soup.findAll('div',{'class':'Normal'})
            corpus_dict[key] = review_texts

        except:
            corpus_dict[key] = ""
            sad.append(ur)
            print(" bad link=\n"+ str(ur)+" \n is\n "+ str(review_texts)+"\n\n\n")

#MAKING THREADS IN A RANGE  ####################################

    for i in range(start,stop):
        u = df1.loc[i,"link_post"]
        key = df1.loc[i,"keys"]
        t = Thread(target=th,args=(u,key))
        t.start()
        threadlist.append(t)

    for b in threadlist:
        b.join()

#STORING CORPUS TO THEIR CORRESPONDING LINKS  ##################

    for i in range(start,stop):
        if len(str(corpus_dict[i]))<100:
            sad.append(df1.loc[i,"link_post"])
            
        df1.loc[i,'corpus']=str(corpus_dict[i])
    print("end of search-"+paper)

In [0]:
df1.to_csv('/Users/ipsita.praharaj1997/Desktop/cotton suicides india.csv', index=False)

# Topic modelling of documents

In [0]:
Main_corpus = pd.read_csv('/Users/ipsita.praharaj1997/Desktop/farmer suicides in india - no of searches=1018 .csv')
corpu1= Main_corpus['corpus']

In [0]:
Main_corpus

,Dates,Paper,Head_line,link_post,keys,corpus
0,15-Jun-2018,Zee News हिन्दी,VIDEO: किसानों की आत्महत्या पर योगी के मंत्री ...,http://zeenews.india.com/hindi/india/up-uttara...,0,NaN
1,14-Jun-2018,Zee News हिन्दी,कौशाम्बी: रिश्वतखोरी से तंग आकर किसान ने की ...,http://zeenews.india.com/hindi/india/up-uttara...,1,NaN
2,03-Jun-2018,Zee News,5W1H: 2 farmers suicide in Madhya Pradesh foll...,http://zeenews.india.com/video/india/5w1h-2-fa...,2,"[<p class=""margin-bt10px"">5W1H: This is a segm..."
3,28-Apr-2018,Zee News,"On farmer deaths, MP minister says suicides ar...",http://zeenews.india.com/india/on-farmer-death...,3,"[<p class=""margin-bt10px"">Balkrishna Patidar h..."
4,13-Jun-2018,Zee Business,Top officials not encouraging young entreprene...,http://www.zeebiz.com/india/news-top-officials...,4,[<p>Startups and young entrepreneurs in India ...
5,10-Jun-2018,YourStory.com,"""Oncologist, entrepreneur, startup mentor: how...",https://yourstory.com/2018/06/oncologist-entre...,5,"[<p></p>, <p style=""font-weight: 400;""><em>Dr ..."
6,09-Aug-2017,YourStory.com,"""Aruna Roy on Indias agrarian crisis: Policy w...",https://yourstory.com/2017/08/aruna-roy-farmer...,6,"[<p><em>70 years after Independence, India’s f..."
7,23-Aug-2017,YourStory.com,Battling farmer suicides in Vidarbha: an 83-ye...,https://yourstory.com/2017/08/gram-seva-mandal...,7,[Vinoba Bhave founded Gram Seva Mandal back in...
8,10-May-2018,YourStory.com,"As Karnataka readies for new government, Agrit...",https://yourstory.com/2018/05/karnataka-electi...,8,"[As Karnataka goes to the polls on Saturday, Y..."
9,17-Jul-2017,YourStory.com,A scooter trip across four countries to spread...,https://yourstory.com/2017/07/scooter-trip-acr...,9,"[Arunabh Mazumdar embarks on a unique, solo qu..."


In [0]:
%time
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')
more_stopwords=str("farmer http com span itext per strong said text internallink suicide commit farmers suicides india commits crisis killing kill die fail time indias agriculture record amid records crop govt farm agrarian debt life failed loan end old credit shocking epidemic financial year distress every government ")
en_stop.extend(more_stopwords.split(sep=" "))

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# Create lemmetaizer
lmtzr = WordNetLemmatizer()

#Our Document
corpustry = pd.read_csv('/Users/ipsita.praharaj1997/Desktop/farmer suicides in india - no of searches=1018 .csv')
corpu1=corpustry['corpus'].astype(str)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11 µs


In [0]:
len(en_stop)

907

In [0]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in corpu1:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    tokens= list(filter(lambda x: len(x) > 2, tokens))
    

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    #stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    lem_tokens= [lmtzr.lemmatize(w) for w in stopped_tokens]

    
    # add tokens to list
    texts.append(lem_tokens)

In [0]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
print(dictionary )

Dictionary(24294 unique tokens: ['nan', '3rd', '5w1h', 'able', 'aboutads']...)


In [0]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [0]:
%time
#generate LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=50,chunksize=200,update_every=1,iterations=200)

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 15.3 µs


In [0]:
import pyLDAvis.gensim
%time
print(ldamodel.print_topics(num_topics=3, num_words=5))

CPU times: user 11 µs, sys: 2 µs, total: 13 µs
Wall time: 11.9 µs
[(3, '0.010*"font" + 0.009*"href" + 0.008*"style" + 0.006*"color" + 0.005*"target"'), (1, '0.013*"class" + 0.008*"state" + 0.007*"family" + 0.007*"href" + 0.006*"land"'), (2, '0.053*"2018" + 0.040*"ist" + 0.033*"class" + 0.022*"title" + 0.021*"july"')]


In [0]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
news = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary)

In [0]:
news

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      31.520239        1       1 -0.097020  0.092529
1      24.189333        1       2 -0.069205  0.137710
2      17.083910        1       3  0.299698  0.050349
0      15.274983        1       4  0.001107 -0.253109
4      11.931526        1       5 -0.134580 -0.027480, topic_info=      Category         Freq              Term        Total  loglift  logprob
term                                                                        
301    Default  5696.000000              2018  5696.000000  30.0000  30.0000
13186  Default  3654.000000               ist  3654.000000  29.0000  29.0000
51     Default  7847.000000             class  7847.000000  28.0000  28.0000
3581   Default  2027.000000              july  2027.000000  27.0000  27.0000
2229   Default  2607.000000             title  2607.000000  26.0000  26.0000
2298   Default  1674.000000           updated  1674.000000  25.0000  25.0000
13306  Default  1587.000000             ltext  1587.000000  24.0000  24.0000
1575   Default  2186.000000             woman  2186.000000  23.0000  23.0000
2439   Default  1807.000000         published  1807.000000  22.0000  22.0000
63     Default  1581.000000           content  1581.000000  21.0000  21.0000
293    Default  3929.000000               www  3929.000000  20.0000  20.0000
342    Default  3603.000000              http  3603.000000  19.0000  19.0000
247    Default  2237.000000             style  2237.000000  18.0000  18.0000
105    Default  1686.000000              font  1686.000000  17.0000  17.0000
118    Default  6272.000000              href  6272.000000  16.0000  16.0000
3132   Default  1005.000000           cricket  1005.000000  15.0000  15.0000
13573  Default   890.000000  financialexpress   890.000000  14.0000  14.0000
13207  Default  1063.000000             score  1063.000000  13.0000  13.0000
1505   Default  1274.000000               rel  1274.000000  12.0000  12.0000
3973   Default   833.000000              meta   833.000000  11.0000  11.0000
910    Default  1215.000000              live  1215.000000  10.0000  10.0000
484    Default  1398.000000              june  1398.000000   9.0000   9.0000
1518   Default   728.000000            script   728.000000   8.0000   8.0000
6220   Default  1044.000000             color  1044.000000   7.0000   7.0000
1811   Default   727.000000              type   727.000000   6.0000   6.0000
241    Default  2732.000000             state  2732.000000   5.0000   5.0000
1244   Default  2145.000000            _blank  2145.000000   4.0000   4.0000
4629   Default   691.000000           justify   691.000000   3.0000   3.0000
10955  Default   506.000000               div   506.000000   2.0000   2.0000
20061  Default   604.000000      firstcricket   604.000000   1.0000   1.0000
...        ...          ...               ...          ...      ...      ...
136     Topic5   154.547653            latest   226.816727   1.7423  -6.0207
233     Topic5   309.212311             share   588.128052   1.4831  -5.3271
8322    Topic5    99.474220          fadnavis   127.577698   1.8772  -6.4613
371     Topic5   445.991150          minister  1052.522949   1.2673  -4.9609
2341    Topic5   163.558365             court   256.706238   1.6752  -5.9640
241     Topic5   820.455627             state  2732.528809   0.9229  -4.3513
243     Topic5   106.723412             store   141.662537   1.8428  -6.3909
1384    Topic5   198.490372           hectare   360.702057   1.5287  -5.7704
3864    Topic5   283.696075          congress   653.381714   1.2917  -5.4133
26      Topic5   390.577942              also  1403.140625   0.8471  -5.0935
75      Topic5   374.984711              data  1382.126953   0.8215  -5.1343
1648    Topic5   298.212006          district   926.955811   0.9919  -5.3634
342     Topic5   595.517822              http  3603.135986   0.3259  -4.6717
1446    Topic5   263.960693          

# Collecting all  Topic words 

In [0]:
x=ldamodel.show_topics(num_topics=5, num_words=30,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words 
# for topic,words in topics_words[0]:
#     print(" ".join(words))

for words in topics_words[0:2]:
    en_stop.append(words)
    


0::['style', 'font', 'href', 'www', 'color', 'align', 'class', 'justify', 'jpg', 'uploads', '000000', 'content', '2017', '400', 'weight', 'http', 'org', 'size', '_blank', 'indiaspend', 'target', 'rate', 'width', 'indian', 'height', 'story', '3366ff', '2018', 'news', 'available']
1::['2018', 'ist', 'class', 'title', 'july', 'woman', 'updated', 'published', 'href', 'ltext', 'june', 'live', 'cricket', 'score', 'www', 'html', 'firstcricket', 'full', 'series', 'jul', 'world', 'may', 't20i', 'newindianexpress', 'news', 'twenty20', 'icc', 'qualifier', 't20', '3019']
2::['class', 'family', 'href', 'land', 'farming', 'like', '000', 'one', 'pesticide', 'village', 'www', 'will', 'target', 'can', 'html', 'http', 'also', '_blank', 'article', 'cotton', 'price', 'people', 'many', 'now', 'day', 'indian', 'get', 'jazeera', 'crop', 'bank']
3::['state', 'href', 'agricultural', 'itemprop', 'target', 'news', 'climate', '_blank', 'articlebody', 'number', 'country', '2015', 'drought', 'change', 'data', 'cent

# Storing the probability distrubution

In [0]:
topic_a=[]
topic_b=[]
topic_c=[]
topic_d=[]
topic_e=[]

for i in range(len(corpus)):
#     print('doc id:'+str(i))
    try:
        topic_a.append(ldamodel[corpus[i]][0])
    except:
        topic_a.append(0)
    try:
        topic_b.append(ldamodel[corpus[i]][1])
    except:
        topic_b.append(0)
    try:
        topic_c.append(ldamodel[corpus[i]][2])
    except:
        topic_c.append(0)
    try:
        topic_d.append(ldamodel[corpus[i]][3])
    except:
        topic_d.append(0)
    try:
        topic_e.append(ldamodel[corpus[i]][4])
    except:
        topic_e.append(0)
    
    i=i+1
# len(topic_d  )
doc_dis = pd.DataFrame( { "key": corpustry['keys'], "topic_0":topic_a, "topic_1":topic_b, "topic_2":topic_c, "topic_3":topic_d, "topic_4":topic_e } )

In [0]:
doc_dis.to_csv('/Users/ipsita.praharaj1997/Desktop/lda-farmer suicides in india - no of searches=1018.csv')

# Narrowing down to topics

Maharastra emerges as the highest used token in the most prevalent topic -1, therefore we arrange the document in the order of topic-1 and re run the model  with a threshold of 0.66

In [0]:
#Our Document
corpustry = pd.read_csv('/Users/ipsita.praharaj1997/Desktop/lda-2.csv')
keys=corpustry.loc[:186,'key'].astype(str)


In [0]:
corpu2={}

for i in range(len(corpu1)):
        if(i in keys):
            corpu2[i]=corpu1[i]


In [0]:
# create English stop words list
en_stop = get_stop_words('en')
more_stopwords=str("farmer say find percent waiver maharashtra suicide commit farmers suicides india commits crisis killing kill die fail time indias agriculture record amid records crop govt farm agrarian debt life failed loan end old credit shocking epidemic financial year distress every government ")
en_stop.extend(more_stopwords.split(sep=" "))




# list for tokenized documents in loop
texts = []

# loop through document list
for i in corpu2.values():
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    tokens= list(filter(lambda x: len(x) > 2, tokens))

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    #Lemmatizing the tokens
    lem_tokens= [lmtzr.lemmatize(w) for w in stopped_tokens]

     # add tokens to list
    texts.append(lem_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

%time
#generate LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=50,chunksize=200,update_every=1,iterations=100)

In [0]:
news2 = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary)
news2

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      24.098419        1       1  0.133762 -0.050452
4      21.695175        1       2 -0.076363 -0.127739
3      20.629606        1       3 -0.071903  0.048099
2      17.100643        1       4  0.046388  0.058775
0      16.476158        1       5 -0.031884  0.071317, topic_info=     Category      Freq         Term     Total  loglift  logprob
term                                                            
115   Default  4.000000      climate  4.000000  30.0000  30.0000
216   Default  4.000000          bjp  4.000000  29.0000  29.0000
11    Default  4.000000     minister  4.000000  28.0000  28.0000
62    Default  3.000000       matter  3.000000  27.0000  27.0000
143   Default  4.000000          day  4.000000  26.0000  26.0000
158   Default  3.000000         food  3.000000  25.0000  25.0000
465   Default  2.000000        sunil  2.000000  24.0000  24.0000
462   Default  2.000000       jakhar  2.000000  23.0000  23.0000
463   Default  2.000000         slam  2.000000  22.0000  22.0000
464   Default  2.000000     speaking  2.000000  21.0000  21.0000
458   Default  2.000000          bku  2.000000  20.0000  20.0000
459   Default  2.000000        block  2.000000  19.0000  19.0000
460   Default  2.000000      justice  2.000000  18.0000  18.0000
461   Default  2.000000         road  2.000000  17.0000  17.0000
144   Default  2.000000        dists  2.000000  16.0000  16.0000
145   Default  2.000000  marathawada  2.000000  15.0000  15.0000
422   Default  2.000000      product  2.000000  14.0000  14.0000
55    Default  2.000000    scientist  2.000000  13.0000  13.0000
54    Default  2.000000       method  2.000000  12.0000  12.0000
53    Default  2.000000     fighting  2.000000  11.0000  11.0000
52    Default  2.000000         cost  2.000000  10.0000  10.0000
56    Default  2.000000         zero  2.000000   9.0000   9.0000
25    Default  3.000000      startup  3.000000   8.0000   8.0000
175   Default  2.000000    agitation  2.000000   7.0000   7.0000
118   Default  3.000000       health  3.000000   6.0000   6.0000
142   Default  3.000000        since  3.000000   5.0000   5.0000
51    Default  3.000000         save  3.000000   4.0000   4.0000
65    Default  2.000000      sainath  2.000000   3.0000   3.0000
64    Default  2.000000   population  2.000000   2.0000   2.0000
63    Default  2.000000       medium  2.000000   1.0000   1.0000
...       ...       ...          ...       ...      ...      ...
92     Topic5  0.694577          way  1.193006   1.2623  -5.2595
503    Topic5  0.694577          pre  1.193006   1.2623  -5.2595
502    Topic5  0.694577      meeting  1.193006   1.2623  -5.2595
501    Topic5  0.694577       demand  1.193006   1.2623  -5.2595
122    Topic5  0.694577        didnt  1.193006   1.2623  -5.2595
124    Topic5  0.694577        still  1.193006   1.2623  -5.2595
125    Topic5  0.694577       worked  1.193006   1.2623  -5.2595
359    Topic5  0.694575  association  1.193007   1.2623  -5.2595
360    Topic5  0.694575         file  1.193007   1.2623  -5.2595
361    Topic5  0.694575     petition  1.193007   1.2623  -5.2595
362    Topic5  0.694575       review  1.193007   1.2623  -5.2595
363    Topic5  0.694575      seeking  1.193007   1.2623  -5.2595
332    Topic5  0.694573        blush  1.193007   1.2623  -5.2595
333    Topic5  0.694573        fresh  1.193007   1.2623  -5.2595
136    Topic5  1.859123         last  3.619774   1.1369  -4.2749
85     Topic5  1.274369       income  2.389599   1.1746  -4.6526
123    Topic5  1.278186         modi  3.016406   0.9446  -4.6496
41     Topic5  1.277164         four  3.041327   0.9356  -4.6504
61     Topic5  1.856340       indian  6.727604   0.5156  -4.2764
50     Topic5  1.277257          can  5.624278   0.3209  -4.6503
295    Topic5  0.696522       budget  3.684770   0.1374  -5.2567
182    Topic5  0.695469        month  3.057817   0.3224  -5.2582
14  

 # major keywords: vidarbhas, marathawada, jakhar, bjp, climate, minister